In [ ]:
import random
from typing import TypedDict
from langgraph.graph import StateGraph, END

In [ ]:
class GameState(TypedDict):
    player_name: str
    target_number: int
    attempts: int
    guesses: list[int]
    hint: str
    lower_bound: int
    upper_bound: int

In [ ]:
def start_game(state: GameState) -> GameState:
    """Initialize the game state with a random target number and reset attempts and guesses."""
    
    state['player_name'] = f"Welcome to the game {state['player_name']}!"
    state['target_number'] = random.randint(1, 20)
    state['attempts'] = 0
    state['guesses'] = []
    state['hint'] = "Game started! Try to guess the number between 1 and 20."
    state['lower_bound'] = 1
    state['upper_bound'] = 20
    print(f"{state['player_name']} the game has begun. I'm thinking about of a number between {state['lower_bound']} and {state['upper_bound']}." )
    return state

In [ ]:
def guess_node(state: GameState) -> GameState:
    """Process the player's guess and update the game state accordingly."""
    
    possible_guess = [i for i in range(state['lower_bound'], state['upper_bound'] + 1) if i not in state['guesses']]

    if possible_guess:
        guess = random.choice(possible_guess)
    else:
        guess = random.randint(state['lower_bound'], state['upper_bound'])
    
    state['guesses'].append(guess)
    state['attempts'] += 1
    print(f"Attempt {state['attempts']}: Player guessed {guess}. (Current range: {state['lower_bound']} - {state['upper_bound']})")
    return state

In [ ]:
def hint_node(state: GameState) -> GameState:
    """Provide a hint based on the player's guess."""
    
    latest_guess = state['guesses'][-1]
    if latest_guess == state['target_number']: # Correct guess
        state['hint'] = f"Congratulations! You've guessed the number {state['target_number']} in {state['attempts']} attempts."

    elif latest_guess < state['target_number']: # Smaller Number
        state['hint'] = f"The number {latest_guess} is too low. Try a higher number."
        state["lower_bound"] = max(state["lower_bound"], latest_guess + 1)

        print("Hint: ", state['hint'])
    else: # Larger Number
        state['hint'] = f"The number {latest_guess} is too high. Try a lower number."
        state['upper_bound'] = min(state['upper_bound'], latest_guess)

        print("Hint: ", state['hint'])
    return state

In [ ]:
def should_continue(state: GameState) -> GameState:
    """Determine if we should continue guessing or end the game"""

    #Only 2 conditions. The correct number is guessed or the maximum attemps is reached.add()

    latest_guess = state['guesses'][-1]

    if latest_guess == state['target_number']:
        print("Game Over: You've guessed the number correctly!")
        return "end"
    elif state['attempts'] >= 7:
        print("Game Over: You've reached the maximum number of attempts.")
        return "end"
    else:
        print(f"CONTINUING. {state['attempts']}/7 attempts used")
        return "continue"

In [ ]:
graph = StateGraph(GameState)

#Nodes
graph.add_node("setup", start_game)
graph.add_node("guess", guess_node)
graph.add_node("hint_node", hint_node)

#Edges
graph.add_edge("setup", "guess")
graph.add_edge("guess", "hint_node")

#Conditional Edges
graph.add_conditional_edges(
    "hint_node",
    should_continue,
    {
        "continue": "guess",
        "end": END
    }
)

graph.set_entry_point("setup")

app = graph.compile()

In [ ]:
app

In [ ]:
result = app.invoke({"player_name": "James", "guesses": [], "attempts": 0, "lower_bound": 1, "upper_bound": 20})